In [1]:
import numpy as np
import pandas as pd
import airmass

day_of_year = 355
latitude = 40

In [2]:
als = np.arange(0, 50001, 1000)
hrs = np.arange(51) * 24 / 50
als2, hrs2 = np.meshgrid(als, hrs)
vf = np.vectorize(
    lambda altitude, hour: airmass.solar_intensity_time(altitude, day_of_year=day_of_year, latitude=latitude,
                                                        hour_of_day=hour, include_diffuse_sky=True)
)
mean_insol_grid = vf(als2, hrs2)
mean_insol_grid = pd.DataFrame(mean_insol_grid)
mean_insol_grid.index = hrs * 3600
mean_insol_grid.columns = als

In [3]:
# Reindex index + columns to place consistent NaNs, then interpolate along both axes to fill those NaNs
mean_insol_grid = mean_insol_grid.reindex(set(np.arange(0, 3600.0 * 24 + 1, 60.0)).union(mean_insol_grid.index))
mean_insol_grid = mean_insol_grid.sort_index()
mean_insol_grid = mean_insol_grid.reindex(set(np.arange(0, 50001.0, 25.0)).union(mean_insol_grid.columns), axis=1)
mean_insol_grid = mean_insol_grid.sort_index(axis=1)
mean_insol_grid = mean_insol_grid.interpolate(method='cubic').interpolate(method='cubic', axis=1)
mean_insol_grid.index.name = 'second'

In [4]:
mean_insol_grid.to_csv(f'solar_graph_day_{day_of_year}_lat_{latitude}.csv')